In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import cartopy.crs as ccrs
import numpy as np
import pandas as pd
from scipy.interpolate import splprep, splev
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import metpy.calc as mpcalc
from metpy.units import units
from matplotlib.colors import Normalize
from scipy.ndimage import binary_erosion, generic_filter
import matplotlib.cm as cm
from pandas import DataFrame, Index
import os
from scipy.ndimage import uniform_filter

In [2]:
#predictions NWP#
#initialized at 00
NWP_pred_00_0507 = xr.open_dataset('https://thredds.met.no/thredds/dodsC/meps25epsarchive/2023/07/05/meps_det_2_5km_20230705T00Z.nc')
#initialized at 06#
NWP_pred_06_0507 = xr.open_dataset('https://thredds.met.no/thredds/dodsC/meps25epsarchive/2023/07/05/meps_det_2_5km_20230705T06Z.nc')

#analyises NWP
#time 06
NWP_gt_06_0507 = NWP_pred_06_0507


inputs_POLY_NWP_new = [
    (NWP_pred_00_0507, NWP_gt_06_0507, 6, '05/07/2023 06 UTC'),
]

In [3]:
data_dir_DDM = 'C:/Users/User/Documents/Master_Thesis/DATA/DDM_output'
nc_files = [f for f in os.listdir(data_dir_DDM) if f.endswith('.nc')]
nc_files.sort()

data_DDM = {}
for file_name in nc_files:
    file_path = os.path.join(data_dir_DDM, file_name)
    data_DDM[file_name] = xr.open_dataset(file_path)
    #print(f"Uploaded: {file_name}")

# Predictions DDM
# Initialized at 00
DDM_pred_00_0507 = data_DDM['20230705T00Z.nc']
# Initialized at 06
DDM_pred_06_0507 = data_DDM['20230705T06Z.nc']

#analyises DDM
# Time 06
DDM_gt_06_0507 = DDM_pred_06_0507


inputs_POLY_DDM_new = [
    (DDM_pred_00_0507, DDM_gt_06_0507, 1, '05/07/2023 06 UTC'),
]

In [4]:
def prepare_datasets(data_pred, data_gt, lead_time, lat, lon):

    latitude = data_pred['latitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    longitude = data_pred['longitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    
    # Predictions data
    CUT_CENTRE_Phi_PRED = data_pred['geopotential_pl'][lead_time, :, lat[0]:lat[1], lon[0]:lon[1]]
    Phi_pred_multiple_pl = CUT_CENTRE_Phi_PRED.values

    CUT_CENTRE_T_PRED = data_pred['air_temperature_pl'][lead_time, :, lat[0]:lat[1], lon[0]:lon[1]]
    T_pred_multiple_pl = CUT_CENTRE_T_PRED.values

    CUT_CENTRE_sh_PRED = data_pred['specific_humidity_pl'][lead_time, :, lat[0]:lat[1], lon[0]:lon[1]]
    sh_pred_multiple_pl = CUT_CENTRE_sh_PRED.values

    # Ground-Truth data
    CUT_CENTRE_Phi_GT = data_gt['geopotential_pl'][0, :, lat[0]:lat[1], lon[0]:lon[1]]
    Phi_gt_multiple_pl = CUT_CENTRE_Phi_GT.values

    CUT_CENTRE_T_GT = data_gt['air_temperature_pl'][0, :, lat[0]:lat[1], lon[0]:lon[1]]
    T_gt_multiple_pl = CUT_CENTRE_T_GT.values

    CUT_CENTRE_sh_GT = data_gt['specific_humidity_pl'][0, :, lat[0]:lat[1], lon[0]:lon[1]]
    sh_gt_multiple_pl = CUT_CENTRE_sh_GT.values

    return Phi_pred_multiple_pl, Phi_gt_multiple_pl, T_pred_multiple_pl, T_gt_multiple_pl, sh_pred_multiple_pl, sh_gt_multiple_pl

In [5]:
def prepare_datasets_NWP(data_pred, data_gt, lead_time, lat, lon):

    latitude = data_pred['latitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    longitude = data_pred['longitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    
    # Predictions data
    CUT_CENTRE_w_PRED = data_pred['upward_air_velocity_pl'][lead_time, :, lat[0]:lat[1], lon[0]:lon[1]]
    w_pred_multiple_pl = CUT_CENTRE_w_PRED.values

    # Ground-Truth data
    CUT_CENTRE_w_GT = data_gt['upward_air_velocity_pl'][0, :, lat[0]:lat[1], lon[0]:lon[1]]
    w_gt_multiple_pl = CUT_CENTRE_w_GT.values

    return w_pred_multiple_pl, w_gt_multiple_pl

def prepare_datasets_DDM(data_pred, data_gt, lead_time, lat, lon):

    latitude = data_pred['latitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    longitude = data_pred['longitude'][lat[0]:lat[1], lon[0]:lon[1]].values
    
    # Predictions data
    CUT_CENTRE_w_PRED = data_pred['vertical_velocity_pl'][lead_time, :, lat[0]:lat[1], lon[0]:lon[1]]
    wp_pred_multiple_pl = CUT_CENTRE_w_PRED.values

    # Ground-Truth data
    CUT_CENTRE_w_GT = data_gt['vertical_velocity_pl'][0, :, lat[0]:lat[1], lon[0]:lon[1]]
    wp_gt_multiple_pl = CUT_CENTRE_w_GT.values

    return wp_pred_multiple_pl, wp_gt_multiple_pl

In [6]:
lat_DDM = [7,450]
lon_DDM = [7,450]
lat_NWP = [8,451]
lon_NWP = [8,451]

In [7]:
#DDM
datasets_DDM = []
datasets_wind_DDM = []
    
for i, (data_pred, data_gt, lead_time, time) in enumerate(inputs_POLY_DDM_new):
    Phi_pred_multiple_pl, Phi_gt_multiple_pl, T_pred_multiple_pl, T_gt_multiple_pl, sh_pred_multiple_pl, sh_gt_multiple_pl = prepare_datasets(data_pred, data_gt, lead_time, lat_DDM, lon_DDM)
    wp_pred_multiple_pl, wp_gt_multiple_pl = prepare_datasets_DDM(data_pred, data_gt, lead_time, lat_DDM, lon_DDM)
    
    datasets_DDM.append([Phi_pred_multiple_pl, Phi_gt_multiple_pl, T_pred_multiple_pl, T_gt_multiple_pl, sh_pred_multiple_pl, sh_gt_multiple_pl])
    datasets_wind_DDM.append([wp_pred_multiple_pl, wp_gt_multiple_pl])

datasets_DDM = np.array(datasets_DDM)
datasets_wind_DDM = np.array(datasets_wind_DDM)

In [8]:
#NWP
datasets_NWP = []
datasets_wind_NWP = []
    
for i, (data_pred, data_gt, lead_time, time) in enumerate(inputs_POLY_NWP_new):
    Phi_pred_multiple_pl, Phi_gt_multiple_pl, T_pred_multiple_pl, T_gt_multiple_pl, sh_pred_multiple_pl, sh_gt_multiple_pl = prepare_datasets(data_pred, data_gt, lead_time, lat_NWP, lon_NWP)
    w_pred_multiple_pl, w_gt_multiple_pl = prepare_datasets_NWP(data_pred, data_gt, lead_time, lat_NWP, lon_NWP)
    
    datasets_NWP.append([Phi_pred_multiple_pl, Phi_gt_multiple_pl, T_pred_multiple_pl, T_gt_multiple_pl, sh_pred_multiple_pl, sh_gt_multiple_pl])
    datasets_wind_NWP.append([w_pred_multiple_pl, w_gt_multiple_pl])

datasets_NWP = np.array(datasets_NWP)
datasets_wind_NWP = np.array(datasets_wind_NWP)

In [9]:
p = np.array([1000., 925., 850., 700., 500., 400., 300., 250., 200., 150., 100., 50.])

def convert_wp_in_wz(wp, T, sh, p):
    #p has to be transformed in the same structure of T and sh
    rho = (p * 100) / (287 * T * (1 + 0.61 * sh))
    g = 9.80665
    wz = -wp / (rho*g) 
    return wz

#Prepare DDM datasets
Phi_pred_DDM = datasets_DDM[:, 0, ::-1, :, :] 
Phi_gt_DDM = datasets_DDM[:, 1, ::-1, :, :]   
T_pred_DDM = datasets_DDM[:, 2, ::-1, :, :] 
T_gt_DDM = datasets_DDM[:, 3, ::-1, :, :]   
sh_pred_DDM = datasets_DDM[:, 4, ::-1, :, :] 
sh_gt_DDM = datasets_DDM[:, 5, ::-1, :, :]   
wp_pred_DDM = datasets_wind_DDM[:, 0, ::-1, :, :]
wp_gt_DDM = datasets_wind_DDM[:, 1, ::-1, :, :]
p_4d = p.reshape(1, 12, 1, 1)
w_pred_DDM = convert_wp_in_wz(wp_pred_DDM, T_pred_DDM, sh_pred_DDM, p_4d)
w_gt_DDM = convert_wp_in_wz(wp_gt_DDM, T_gt_DDM, sh_gt_DDM, p_4d)


#Prepare NWP datasets
Phi_pred_NWP = np.delete(datasets_NWP[:, 0, :, :, :], 9, axis=1)
Phi_gt_NWP = np.delete(datasets_NWP[:, 1, :, :, :], 9, axis=1)
T_pred_NWP = np.delete(datasets_NWP[:, 2, :, :, :], 9, axis=1)
T_gt_NWP = np.delete(datasets_NWP[:, 3, :, :, :], 9, axis=1)
sh_pred_NWP = np.delete(datasets_NWP[:, 4, :, :, :], 9, axis=1)
sh_gt_NWP = np.delete(datasets_NWP[:, 5, :, :, :], 9, axis=1)
w_pred_NWP = np.delete(datasets_wind_NWP[:, 0, :, :, :], 9, axis=1)
w_gt_NWP = np.delete(datasets_wind_NWP[:, 1, :, :, :], 9, axis=1)

Phi_pred_NWP = Phi_pred_NWP[:,::-1,:,:]
Phi_gt_NWP = Phi_gt_NWP[:,::-1,:,:]
T_pred_NWP = T_pred_NWP[:,::-1,:,:]
T_gt_NWP = T_gt_NWP[:,::-1,:,:]
sh_pred_NWP = sh_pred_NWP[:,::-1,:,:]
sh_gt_NWP = sh_gt_NWP[:,::-1,:,:]
w_pred_NWP = w_pred_NWP[:,::-1,:,:]
w_gt_NWP = w_gt_NWP[:,::-1,:,:]

In [10]:
def compute_HB_GRAD_dev(inputs_POLY, lat, lon, p, Phi_pred, Phi_gt, T_pred, T_gt, sh_pred, sh_gt):
    
    HB_GRAD_dev_pred = np.zeros_like(Phi_pred)
    HB_GRAD_dev_gt = np.zeros_like(Phi_gt)
    
    for i in range(len(p)-1):
        dPhi_obs_pred = (Phi_pred[:, i+1, :, :] - Phi_pred[:, i, :, :]) 
        dPhi_obs_gt = (Phi_gt[:, i+1, :, :] - Phi_gt[:, i, :, :]) 
        
        p_mid = 0.5 * (p[i+1] + p[i])  
        T_mid_pred = 0.5 * (T_pred[:, i+1, :, :]*(1 + 0.61 * sh_pred[:, i+1, :, :]) + + T_pred[:, i, :, :]*(1 + 0.61 * sh_pred[:, i, :, :]))
        T_mid_gt = 0.5 * (T_gt[:, i+1, :, :]*(1 + 0.61 * sh_gt[:, i+1, :, :]) + T_gt[:, i, :, :]*(1 + 0.61 * sh_gt[:, i, :, :]))
        
        dPhi_hydro_pred = - 287.05 * (T_mid_pred / (p_mid)) * (p[i+1] - p[i])
        dPhi_hydro_gt = - 287.05 * (T_mid_gt / (p_mid)) * (p[i+1] - p[i])
        
        HB_GRAD_dev_pred[:, i, :, :] = (dPhi_obs_pred - dPhi_hydro_pred)*100/dPhi_obs_pred
        HB_GRAD_dev_gt[:, i, :, :] = (dPhi_obs_gt - dPhi_hydro_gt)*100/dPhi_obs_gt
    
    mean_grad_dev_pred = np.mean(np.abs(HB_GRAD_dev_pred), axis=(2,3))
    mean_grad_dev_gt = np.mean(np.abs(HB_GRAD_dev_gt), axis=(2,3))
    
    return HB_GRAD_dev_pred, HB_GRAD_dev_gt, mean_grad_dev_pred, mean_grad_dev_gt


In [11]:
#Calculations down up GRADIENT DEVIATION

#NWP
HB_GRAD_dev_pred_NWP, HB_GRAD_dev_gt_NWP, mean_grad_dev_pred_NWP, mean_grad_dev_gt_NWP = compute_HB_GRAD_dev(inputs_POLY_NWP_new, lat_NWP, lon_NWP, p, Phi_pred_NWP, Phi_gt_NWP, T_pred_NWP, T_gt_NWP, sh_pred_NWP, sh_gt_NWP)

#DDM
HB_GRAD_dev_pred_DDM, HB_GRAD_dev_gt_DDM, mean_grad_dev_pred_DDM, mean_grad_dev_gt_DDM = compute_HB_GRAD_dev(inputs_POLY_DDM_new, lat_DDM, lon_DDM, p, Phi_pred_DDM, Phi_gt_DDM, T_pred_DDM, T_gt_DDM, sh_pred_DDM, sh_gt_DDM)


In [12]:
def plot_balances_distribution_with_isobars_3plots(
    NWP_pred_00_0507,
    DDM_pred_00_0507,
    lat_NWP, lon_NWP,
    lat_DDM, lon_DDM,
    HB_DEVIATION_pred_NWP,
    HB_DEVIATION_pred_DDM,
    HB_DEVIATION_gt_DDM,
    Phi_pred_NWP,
    Phi_pred_DDM,
    Phi_gt_DDM,
    cmap_pred='viridis',
    cmap_gt='viridis',
    norm_pred=None,
    norm_gt=None,
    coastlines=True,
    colorbar=True,
    colorbar_label=True,
    parts_box_text=None,
    titles=None,
    n_contours=10,
    ylabel=None,
    mean_centered=False,  # <-- key switch
    lon_vals=True  # <-- new parameter for longitude ticks
):
    """
    Plot 3-panel balance distribution with isobars, with optional mean centering.

    If mean_centered=True, subtracts mean from each dataset and adds mean value boxes.
    """

    def mean_centering(data):
        return data - np.nanmean(data)

    # Compute means (used in both cases)
    mean_pred_NWP = np.nanmean(HB_DEVIATION_pred_NWP)
    mean_pred_DDM = np.nanmean(HB_DEVIATION_pred_DDM)
    mean_gt_DDM   = np.nanmean(HB_DEVIATION_gt_DDM)

    # Apply mean centering if requested
    if mean_centered:
        HB_DEVIATION_pred_NWP = mean_centering(HB_DEVIATION_pred_NWP)
        HB_DEVIATION_pred_DDM = mean_centering(HB_DEVIATION_pred_DDM)
        HB_DEVIATION_gt_DDM   = mean_centering(HB_DEVIATION_gt_DDM)

    # Extract coordinates
    latitude_NWP = NWP_pred_00_0507['latitude'][lat_NWP[0]:lat_NWP[1], lon_NWP[0]:lon_NWP[1]].values
    longitude_NWP = NWP_pred_00_0507['longitude'][lat_NWP[0]:lat_NWP[1], lon_NWP[0]:lon_NWP[1]].values
    latitude_DDM = DDM_pred_00_0507['latitude'][lat_DDM[0]:lat_DDM[1], lon_DDM[0]:lon_DDM[1]].values
    longitude_DDM = DDM_pred_00_0507['longitude'][lat_DDM[0]:lat_DDM[1], lon_DDM[0]:lon_DDM[1]].values

    # Create figure with fixed aspect ratio
    fig, axes = plt.subplots(
        nrows=1, ncols=3, figsize=(22, 7),
        subplot_kw={'projection': ccrs.PlateCarree()},
        gridspec_kw={'width_ratios': [1, 1, 1], 'wspace': 0.05}
    )

    ax_nwp, ax_ddm, ax_gt = axes

    # Set consistent extent for all plots
    consistent_extent = [-4.1, 16.25, 50.45, 61.65]
    
    # Define tick values
    lat_vals = [51, 53, 55, 57, 59, 61]
    lon_vals_list = [-5, 0, 5, 10, 15]
    
    # Plotting helper
    def plot_panel(ax, lon, lat, data, phi, cmap, norm, title, is_left_panel=False):
        sc = ax.scatter(lon, lat, c=data, cmap=cmap, norm=norm,
                        transform=ccrs.PlateCarree(), s=5)
        if phi is not None:
            contour = ax.contour(lon, lat, phi, n_contours, colors='dodgerblue',
                                 linewidths=1, transform=ccrs.PlateCarree())
            ax.clabel(contour, inline=True, fontsize=14, fmt='%1.0f')
        # Always set the consistent extent
        ax.set_extent(consistent_extent, crs=ccrs.PlateCarree())
        
        # Set latitude ticks on the left panel only
        if is_left_panel:
            ax.set_yticks(lat_vals, crs=ccrs.PlateCarree())
            # Add degree symbol
            ax.set_yticklabels([f"{lat}°" for lat in lat_vals], fontsize=14)
        else:
            ax.set_yticks([])

        # Set longitude ticks if lon_vals is True
        if lon_vals:
            ax.set_xticks(lon_vals_list, crs=ccrs.PlateCarree())
            # Add degree symbol
            ax.set_xticklabels([f"{lon}°" for lon in lon_vals_list], fontsize=14)
        else:
            ax.set_xticks([])
            
        if coastlines:
            ax.coastlines(resolution='10m', color='black', linewidth=0.8)
        if title:
            ax.set_title(title, fontsize=21, pad=15)
        return sc

    # Create plots - mark the left panel
    nwp_plot = plot_panel(ax_nwp, longitude_NWP, latitude_NWP, HB_DEVIATION_pred_NWP, Phi_pred_NWP,
                          cmap_pred, norm_pred, titles[0] if titles else None, is_left_panel=True)

    ddm_plot = plot_panel(ax_ddm, longitude_DDM, latitude_DDM, HB_DEVIATION_pred_DDM, Phi_pred_DDM,
                          cmap_pred, norm_pred, titles[1] if titles else None, is_left_panel=False)

    gt_plot = plot_panel(ax_gt, longitude_DDM, latitude_DDM, HB_DEVIATION_gt_DDM, Phi_gt_DDM,
                         cmap_gt, norm_gt, titles[2] if titles else None, is_left_panel=False)

    # Add mean boxes if mean_centered=True
    if mean_centered and parts_box_text:
        ax_nwp.text(0.01, 0.96, f"{parts_box_text[0]}{mean_pred_NWP:+.2f}{parts_box_text[1]}",
                    transform=ax_nwp.transAxes, fontsize=16, color='white',
                    va='top', ha='left',
                    bbox=dict(facecolor='black', alpha=0.8, boxstyle='round,pad=0.3'))

        ax_ddm.text(0.01, 0.96, f"{parts_box_text[0]}{mean_pred_DDM:+.2f}{parts_box_text[1]}",
                    transform=ax_ddm.transAxes, fontsize=16, color='white',
                    va='top', ha='left',
                    bbox=dict(facecolor='black', alpha=0.8, boxstyle='round,pad=0.3'))

        ax_gt.text(0.01, 0.96, f"{parts_box_text[0]}{mean_gt_DDM:+.2f}{parts_box_text[1]}",
                   transform=ax_gt.transAxes, fontsize=16, color='white',
                   va='top', ha='left',
                   bbox=dict(facecolor='black', alpha=0.8, boxstyle='round,pad=0.3'))

    # Add ylabel (left)
    if ylabel:
        fig.text(0.04, 0.5, ylabel, va='center', rotation=90, fontsize=20)

    # Add colorbar
    if colorbar:
        cbar_ax = fig.add_axes([0.91, 0.3, 0.01, 0.45])
        cbar = fig.colorbar(gt_plot, cax=cbar_ax, orientation='vertical')

        # Format ticks to always show 2 decimal places
        cbar.formatter = FormatStrFormatter('%.2f')
        cbar.update_ticks()
    
    if colorbar_label:
        cbar.set_label(colorbar_label, fontsize=17)
        
    # Make tick labels larger
    cbar.ax.tick_params(labelsize=14)  # Increase from default size

    plt.subplots_adjust(top=0.90, bottom=0.15, left=0.08)
    return fig  # Return figure for saving

In [13]:
lat = [0,443]
lon = [0,443]
LL = 0

# Create a list to store all figures
all_figures = []

##############################################
global_min = -0.2
global_max = +0.2
norm_geo_bal = plt.Normalize(vmin=global_min, vmax=global_max)
cmap_geo_bal = plt.get_cmap('RdBu_r')
fig1 = plot_balances_distribution_with_isobars_3plots(
    NWP_pred_00_0507, DDM_pred_00_0507,
    lat_NWP, lon_NWP, lat_DDM, lon_DDM,
    HB_GRAD_dev_pred_NWP[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    HB_GRAD_dev_pred_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    HB_GRAD_dev_gt_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    None,
    None,
    None,
    cmap_pred=cmap_geo_bal, cmap_gt=cmap_geo_bal,
    norm_pred=norm_geo_bal, norm_gt=norm_geo_bal,
    colorbar_label=r'$\delta_{\mathrm{HB}} - \langle\delta_{\mathrm{HB}}\rangle$ (%)',
    parts_box_text=[r"$\langle \delta_{\mathrm{HB}} \rangle = ", r"\,\%$"],
    ylabel=r"$\delta_{\mathrm{HB}} - \langle\delta_{\mathrm{HB}}\rangle$",
    titles = None,
    mean_centered=True,
    lon_vals = True
)
all_figures.append(fig1)

##################################
global_min = - 0.1
global_max = + 0.1

norm_geo_bal = plt.Normalize(vmin=global_min, vmax=global_max)
cmap_geo_bal = plt.get_cmap('Spectral_r')
fig2 = plot_balances_distribution_with_isobars_3plots(
    NWP_pred_00_0507, DDM_pred_00_0507,
    lat_NWP, lon_NWP, lat_DDM, lon_DDM,
    w_pred_NWP[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    w_pred_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    w_gt_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    None,
    None,
    None,
    cmap_pred=cmap_geo_bal, cmap_gt=cmap_geo_bal,
    norm_pred=norm_geo_bal, norm_gt=norm_geo_bal,
    colorbar_label=r'w (m/s)',
    parts_box_text=[r"", r""],
    ylabel=r"w",
    titles = None,
    mean_centered=False, 
    lon_vals = False
)
all_figures.append(fig2)

##############################################
global_min = - 7.5
global_max = + 7.5

norm_geo_bal = plt.Normalize(vmin=global_min, vmax=global_max)
cmap_geo_bal = plt.get_cmap('seismic')
fig3 = plot_balances_distribution_with_isobars_3plots(
    NWP_pred_00_0507, DDM_pred_00_0507,
    lat_NWP, lon_NWP, lat_DDM, lon_DDM,
    T_pred_NWP[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    T_pred_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    T_gt_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]],
    None,
    None,
    None,
    cmap_pred=cmap_geo_bal, cmap_gt=cmap_geo_bal,
    norm_pred=norm_geo_bal, norm_gt=norm_geo_bal,
    colorbar_label=r'$T - \langle T \rangle$ (K)',
    parts_box_text=[r"$\langle T \rangle = ", r"\,K$"],
    ylabel=r"$T - \langle T \rangle$",
    titles = None,
    mean_centered=True,
    lon_vals = False,
)
all_figures.append(fig3)

##############################################
global_min = 0
global_max = 2.5

norm_geo_bal = plt.Normalize(vmin=global_min, vmax=global_max)
cmap_geo_bal = plt.get_cmap('pink_r')
fig4 = plot_balances_distribution_with_isobars_3plots(
    NWP_pred_00_0507, DDM_pred_00_0507,
    lat_NWP, lon_NWP, lat_DDM, lon_DDM,
    sh_pred_NWP[LL,4,lat[0]:lat[1],lon[0]:lon[1]]*1e3,
    sh_pred_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]]*1e3,
    sh_gt_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]]*1e3,
    Phi_pred_NWP[LL,4,lat[0]:lat[1],lon[0]:lon[1]]/1e2,
    Phi_pred_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]]/1e2,
    Phi_gt_DDM[LL,4,lat[0]:lat[1],lon[0]:lon[1]]/1e2,
    cmap_pred=cmap_geo_bal, cmap_gt=cmap_geo_bal,
    norm_pred=norm_geo_bal, norm_gt=norm_geo_bal,
    colorbar_label=r'$q$ (g/kg)',
    parts_box_text=[r"$\langle q \rangle = ", r"\,(g/kg)$"],
    ylabel=r"$q$",
    titles = ["NWP (lt = 6h)", "DDM (lt = 6h)", "Analysis"],
    mean_centered=False,
    lon_vals = False
)
all_figures.append(fig4)

# Save all figures separately
separate_save_paths = []
for i, fig in enumerate(all_figures):
    # Define what each figure represents for the filename
    if i == 0:
        variable_name = "HB"
    elif i == 1:
        variable_name = "w"
    elif i == 2:
        variable_name = "T"
    elif i == 3:
        variable_name = "q"
    
    # Create save path for individual figure
    separate_save_path = rf'[...]\Fig3_{variable_name}.png'
    
    # Save the individual figure
    fig.savefig(separate_save_path, dpi=300, bbox_inches='tight', pad_inches=0.1)
    separate_save_paths.append(separate_save_path)
    print(f"Figure {i+1} saved as: {separate_save_path}")
    
    # Close the individual figure to free memory
    plt.close(fig)

print("\nAll individual figures saved separately!")

Figure 1 saved as: C:\Users\User\Documents\Paper_KNMI\Images_for_paper_high_resolution_adjusted\Fig3_HB.png
Figure 2 saved as: C:\Users\User\Documents\Paper_KNMI\Images_for_paper_high_resolution_adjusted\Fig3_w.png
Figure 3 saved as: C:\Users\User\Documents\Paper_KNMI\Images_for_paper_high_resolution_adjusted\Fig3_T.png
Figure 4 saved as: C:\Users\User\Documents\Paper_KNMI\Images_for_paper_high_resolution_adjusted\Fig3_q.png

All individual figures saved separately!
